In [1]:
import pandas_datareader as pdr
import datetime as dt
from dateutil.relativedelta import relativedelta

start = dt.datetime.now() - relativedelta(years=1)
end = dt.datetime.now()
btc = pdr.DataReader('BTC-USD', 'yahoo', start, end)

In [2]:
import functools
import pandas_ta as pta
import math

class Strategy:
    def __init__(self, prices, interval):
        self.prices = prices.drop(prices.index[[i for i in range(len(prices.T.columns)) if i % interval != 0]])
        self.spots = []
        self.buying_spots =[]
        self.selling_spots = []
        self.fake_spots = []

    def build(self):
        pass

    def result(self):
        buying_closes = Strategy.getCloses(self.buying_spots)
        selling_closes = Strategy.getCloses(self.selling_spots)
        min_len = len(selling_closes)  if len(buying_closes) > len(selling_closes) else len(buying_closes)
        ratios = [selling_closes[i] / buying_closes[i] for i in range(min_len)]
        return functools.reduce(lambda x, y: x * y, ratios)
        
    def appendSpot(self, array, i):
        return array.append([btc.index[i], btc.Close[i]])

    def getDates(spots):
        return [a[0] for a in spots]

    def getCloses(spots):
        return [a[1] for a in spots]

    def getSpots(self):
        return {
            'spots': self.spots,
            'buying_spots': self.buying_spots,
            'selling_spots': self.selling_spots,
            'fake_spots': self.fake_spots,
        }


class DawnStrategy(Strategy): 
    def __init__(self, prices, interval, period):
        Strategy.__init__(self, prices, interval)
        self.prices["Rsi"] = pta.rsi(self.prices.Close, length=period)
        self.prices["Ema"] = pta.ema(self.prices["Rsi"], length=period)

    def build(self):
        inRange = False

        for i, v in enumerate(btc.Rsi):
            if i > 0:
                current_i = i - 1
                next_i = i
                
                if btc.Rsi[current_i] > btc.Ema[current_i]:
                    entered_in_range_now = False
                    if inRange:
                        self.appendSpot(self.spots, current_i)
                        if btc.Rsi[next_i] < btc.Ema[next_i]:
                            self.appendSpot(self.selling_spots, current_i)
                            inRange = False
                            if entered_in_range_now:
                                self.appendSpot(self.fake_spots, current_i)
                    else: 
                        self.appendSpot(self.buying_spots, current_i)
                        inRange = True
                        entered_in_range_now = True
                        

class AllInStrategy(Strategy): 
    def __init__(self, prices):
        Strategy.__init__(self, prices, 7)

    def build(self):
        self.appendSpot(self.buying_spots, 0)
        self.appendSpot(self.selling_spots, len(btc.Close) - 1)



In [3]:
allInStrategy = AllInStrategy(btc)
allInStrategy.build()
print(allInStrategy.result())

dawnStrategyBtc = DawnStrategy(btc, 7, 14)
dawnStrategyBtc.build()
print(dawnStrategyBtc.result())


0.8065845122592666


AttributeError: 'DataFrame' object has no attribute 'Rsi'

In [127]:
import plotly.offline as py
import plotly.graph_objs as go
import math

def getScatter(spots, color):
    print(spots)
    return go.Scatter(x=Strategy.getDates(spots), y=Strategy.getCloses(spots), mode='markers', marker=dict(size=5, color=color)),

py.init_notebook_mode(connected=True)

py.iplot(
    go.Figure(
        data=[
            # go.Candlestick(x=btc.index, open=btc.Open, high=btc.High, low=btc.Low, close=btc.Close), 
            go.Line(x=btc.index, y=[math.nan if math.isnan(btc.Rsi[i]) else c for i, c in enumerate(btc.Close)]),
            getScatter(dawnStrategyBtc.spots, 'green'),
            getScatter(dawnStrategyBtc.buying_spots, 'blue'),
            getScatter(dawnStrategyBtc.selling_spots, 'red'),
            getScatter(dawnStrategyBtc.fake_spots, 'violet'),
        ],
        layout=go.Layout(title='Bitcoin Candlestick with Range Slider', yaxis={'type': 'log'}, xaxis={'rangeslider':{'visible':False}})), 
    filename='bitcoin_candlestick')


import plotly.express as px

px.line(btc, y=["Rsi", "Ema"], title="Bitcoin RSI and EMA").show()

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [(Scatter({
    'marker': {'color': 'green', 'size': 5},
    'mode': 'markers',
    'x': [2019-03-06 00:00:00, 2019-03-07 00:00:00, 2019-03-08 00:00:00, ...,
          2022-02-14 00:00:00, 2022-02-15 00:00:00, 2022-02-16 00:00:00],
    'y': [3903.942626953125, 3911.484375, 3901.131591796875, ..., 42586.91796875,
          44575.203125, 43961.859375]
}),), (Scatter({
    'marker': {'color': 'blue', 'size': 5},
    'mode': 'markers',
    'x': [2019-03-05 00:00:00, 2019-03-23 00:00:00, 2019-04-22 00:00:00,
          2019-05-02 00:00:00, 2019-05-26 00:00:00, 2019-06-14 00:00:00, 2019-07-08
          00:00:00, 2019-07-31 00:00:00, 2019-08-11 00:00:00, 2019-09-02 00:00:00,
          2019-09-12 00:00:00, 2019-10-07 00:00:00, 2019-10-17 00:00:00, 2019-10-25
          00:00:00, 2019-11-27 00:00:00, 2019-12-05 00:00:00, 2019-12-18 00:00:00,
          2020-01-03 00:00:00, 2020-01-21 00:00:00

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
import math

py.init_notebook_mode(connected=True)

py.iplot(
    go.Figure(
        data=[
            go.Line(x=btc.index, y=btc.Close),
            go.Scatter(x=Strategy.getDates(allInStrategy.buying_spots), y=Strategy.getCloses(allInStrategy.buying_spots), mode='markers', marker=dict(size=5, color='blue')),
            go.Scatter(x=Strategy.getDates(allInStrategy.selling_spots), y=Strategy.getCloses(allInStrategy.selling_spots), mode='markers', marker=dict(size=5, color='red')),
        ],
        layout=go.Layout(title='Bitcoin Candlestick with Range Slider', yaxis={'type': 'log'}, xaxis={'rangeslider':{'visible':False}})), 
    filename='bitcoin_candlestick')


C:\Users\hospicej\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


